# The error matrix

Python activities to complement [*Measurements and their Uncertainties*](https://www.oupcanada.com/catalog/9780199566334.html) (*MU*), Chapter 7, "Computer minimization and the error matrix"

* [Preliminaries](#Preliminaries)
* [The error surface](#The-error-surface)
    * [Example of a linear fit](#Example-of-a-linear-fit)
* [Curvature and covariance](#Curvature-and-covariance)
    * [Interpreting confidence limits](#Interpreting-confidence-limits)
    * [Exercise 1](#Exercise-1)
* [The error matrix for a linear fit](#The-error-matrix-for-a-linear-fit)
* [Summary](#Summary)

## Preliminaries
Before proceeding with this notebook you should review the topics from the [Example Fit notebook](A.1-Example-Fit.ipynb) and read *MU* Ch. 7, "Computer minimization and the error matrix", with the following [goals](A.0-Reading-goals.ipynb#Computer-minimization-and-the-error-matrix) in mind.

1. Be able to explain qualitatively how data analysis computer programs fit a model to data by minimizing the *&chi;*<sup>2</sup> goodness-of-fit parameter as a function of the model parameters. Specifically,
    * recognize that the terms *grid search*, *gradient-descent*, *Newton's method*, *Gauss-Newton*, and *Levenberg-Marquardt* refer to different algorithms for minimizing a function;
    * be able to use matrix notation to expand *&chi;*<sup>2</sup> to second order around a particular point in space, as shown in (7.6); and
    * be able to write the gradient vector, Hessian matrix, and Jacobian matrix for a function of multiple variables, and explain how they appear in the context of computer minimization routines.
2. Be able to describe how the curvature matrix relates to the error surface near $\chi^2_{\min}$, and how it can be used to estimate the parameter values at which $\chi^2=\chi_{\text{min}}^2+1$.
3. Be able to describe the meaning and significance of the covariance matrix and the correlation matrix.
4. Recognize that the covariance matrix can be estimated from a fit by inverting the curvature matrix at the minimum of the error surface.

The following code cell includes the initialization commands needed for this notebook.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import default_rng
from scipy.optimize import curve_fit
from scipy.stats import chi2

%matplotlib inline

## The error surface

We saw in [NB 6.0, Fitting complex functions](6.0-Fitting-complex-functions.ipynb#Linear-fits-with-non-uniform-errors) that to fit a line $y = mx + c$ to measurements $\{x_k\}, \{y_k\}$ with uncertainties $\{\alpha_k\}$, $k = 1, 2, \ldots, N$, we can minimize the function

$$
\chi^2(m,c;\{x_k\},\{y_k\},\{\alpha_k\}) = \sum_{k=1}^N\frac{[y_k - (mx_k +  c)]^2}{\alpha_k^2} \label{eq:lsq}\tag{1}
$$

with respect to the parameters $m$ and $c$. We use a semicolon to emphasize the distinction between the *model parameters*, $m$ and $c$, and the *measurements*, $\{x_k\},\{y_k\}$, and $\{\alpha_k\}$. Typically, we are interested only in the functional dependence of $\chi^2$ on the model parameters, holding the measurement variables fixed.

### Example of a linear fit
The code cell below reproduces *MU* Fig.&nbsp;6.9, which shows the contours of this function for the measurements shown in *MU* Fig.&nbsp;6.1(d). The data are taken from *MU* Exercise (6.1), which is shown in *MU* Fig. 6.1(d) and discussed in *MU* Sec. 7.4.1, "Worked example 1—a straight-line fit." The [Example Fit](A.1-Example-Fit.ipynb) notebook uses the same data.

We also show the $1\sigma$ and $2\sigma$ uncertainty bounds as blue dotted lines, which *circumscribe* the elliptical contours corresponding to $\chi_{1\sigma}^2 = \chi^2_\text{min} + 1$ and $\chi_{2\sigma}^2 = \chi^2_\text{min} + 4$, respectively. As discussed in *MU* Sec. 6.5.2, the reason for this is that we define the uncertainty bounds on $\hat{m}$ to be *independent* of the value of $\hat{c}$; similarly, the uncertainty bounds on $\hat{c}$ are understood to be independent of the value of $\hat{m}$.

In [ ]:
# Load data into arrays
frequency, voltage, err = np.genfromtxt("data/Example-Data.csv", delimiter=",", skip_header=1, unpack=True)

# Define a linear model


def model(m, c):
    return m * frequency + c


# Define the chi-squared function for m and c, given the data


def chi2fun(m, c):
    normres = (voltage - model(m, c)) / err
    return np.sum(normres**2)


# Compute chi-squared over a 2D grid of equally-spaced values of m and c
Nm = 50
Nc = 50
m = np.linspace(1.85, 2.19, num=Nm)
c = np.linspace(-9, 9, num=Nc)

chi2grid = np.zeros([Nc, Nm])
for i in range(Nc):
    for j in range(Nm):
        chi2grid[i, j] = chi2fun(m[j], c[i])

# Fit a linear model to the data and assign results to new variables
pHat, pCov = np.polyfit(frequency, voltage, 1, w=1 / err, cov="unscaled")

mHat = pHat[0]
cHat = pHat[1]
mAlpha = np.sqrt(pCov[0, 0])
cAlpha = np.sqrt(pCov[1, 1])

# Display formatted results
print(f"Model slope (mV/Hz):     {mHat:.2f} ± {mAlpha:.2f}")
print(f"Model intercept (mV):    {cHat:.0f} ± {cAlpha:.0f}")
print()

# Evaluate the chi-squared function at the minimum and define contour levels
#    level_low: standard 1D confidence levels (see caption of MU Fig. 6.9)
#               (Note that we convert the list to a NumPy array to make it clear
#               that the "+" operator represents arithmetic addition, not list
#               concatenation.)
#    level_high: additional contours to show behavior far from minimum
chi2min = chi2fun(mHat, cHat)
level_low = chi2min + np.array([1, 2.71, 4, 9])
level_high = chi2min + np.arange(11, 121, 10)

# Show minimum and contours around minimum
plt.plot(mHat, cHat, "h")
plt.contour(m, c, chi2grid, level_low, colors="k", linewidths=1, linestyles=["solid", "dashed", "dotted", "dotted"])
plt.contour(m, c, chi2grid, level_high, linewidths=0.5)

# Show 1-sigma and 2-sigma uncertainty bounds as dashed lines
plt.hlines(cHat + cAlpha * np.array([-2, -1, 1, 2]), m[0], m[-1], linewidths=1, linestyles="dotted")
plt.vlines(mHat + mAlpha * np.array([-2, -1, 1, 2]), c[0], c[-1], linewidths=1, linestyles="dotted")

# Format plot to resemble MU Fig. 6.9
plt.xticks(np.arange(1.9, 2.19, 0.05))
plt.yticks(np.arange(-9, 10, 3))
plt.xlabel("Gradient (mV/Hz)")
plt.ylabel("Intercept (mV)")
plt.tick_params(direction="in", top=True, right=True)
xmin, xmax, ymin, ymax = plt.axis()
plt.gca().set_aspect(0.8 * (xmax - xmin) / (ymax - ymin))

plt.show()

### Curvature and covariance

To see the dependence of $\chi^2$ on $m$ and $c$ more clearly, we can rewrite Eq.&nbsp;([1](#The-error-surface)) in the form

$$
\chi^2(m, c) = A_{mm}(m - \hat{m})^2 + 2A_{mc}(m - \hat{m})(c - \hat{c}) + A_{cc}(c - \hat{c})^2 + \chi^2_\text{min},\label{eq:ellipse}\tag{2}
$$

where $\chi^2, A_{mm}, A_{mc}, A_{cc}, \hat{m}, \hat{c}$, and $\chi^2_\text{min}$ all depend implicitly on the measurement variables $\{x_k\}, \{y_k\}$, and $\{\alpha_k\}$. This is the most general 2nd-order polynomial expression in $m$ and $c$ that we can write, so there is no need to derive Eq.&nbsp;(\ref{eq:ellipse}) from Eq.&nbsp;(1); we can simply assert their equivalence and use it relate the parameters in Eq.&nbsp;(\ref{eq:ellipse}) to those in Eq.&nbsp;(1). For example, by taking second derivatives of both expressions we can obtain

\begin{align*}
A_{mm} &= \frac{1}{2}\frac{\partial^2(\chi^2)}{\partial m^2} = \sum_{k=1}^{N}\frac{x_k^2}{\alpha_k^2},\label{eq:Amm}\tag{3}\\
A_{mc} &= \frac{1}{2}\frac{\partial^2(\chi^2)}{\partial m\partial c} = \sum_{k=1}^{N}\frac{x_k}{\alpha_k^2},\label{eq:Amc}\tag{4}\\
A_{cc} &= \frac{1}{2}\frac{\partial^2(\chi^2)}{\partial c^2} = \sum_{k=1}^{N}\frac{1}{\alpha_k^2}.\label{eq:Acc}\tag{5}
\end{align*}

We can also see immediately that the minimum of $\chi^2$ is $\chi^2_\text{min} = \chi^2(\hat{m}, \hat{c})$, and we have already derived expressions for $\hat{m}$ and $\hat{c}$ in [NB 6.0](6.0-Fitting-complex-functions.ipynb#Explicit-formulas).

Each contour of constant $\chi^2$ in Eq.&nbsp;(\ref{eq:ellipse}) defines an [ellipse](https://en.wikipedia.org/wiki/Ellipse#General_ellipse) centered on $(\hat{m},\hat{c})$, with its shape determined by $A_{mm}, A_{mc}$, and $A_{cc}$. Writing Eq.&nbsp;(\ref{eq:ellipse}) in matrix form,

\begin{align*}
\chi^2(m,c) - \chi^2_\text{min} &= \begin{bmatrix}(m - \hat{m}) & (c - \hat{c})\end{bmatrix}\begin{bmatrix}A_{mm} & A_{mc}\\ A_{mc} & A_{cc} \end{bmatrix}\begin{bmatrix} m - \hat{m} \\ c - \hat{c}\end{bmatrix},
\end{align*}

and denoting the matrix more compactly as $\mathbf{A}$, we can use the [principal axis theorem](https://en.wikipedia.org/wiki/Principal_axis_theorem) to show that the eigenvectors of $\mathbf{A}$ are along the principal axes of the ellipse, and that the eccentricity of the ellipse is $e = \sqrt{\lambda_1/\lambda_2}$, where $\lambda_1$ and $\lambda_2$ are the eigenvalues of $\mathbf{A}$.

As discussed in *MU* Sec.&nbsp;7.2.1, the covariance matrix $\mathbf{C}$ is the inverse of the curvature matrix,

$$
\mathbf{C} = \mathbf{A}^{-1},
$$

from which we can determine the parameter uncertainties

\begin{align}
\alpha_{\hat{m}} &= \sqrt{C_{mm}}, & \alpha_{\hat{c}} &= \sqrt{C_{cc}}
\end{align}

and the correlation coefficient

$$
\rho_{mc} = \frac{C_{mc}}{\sqrt{C_{mm}C_{cc}}}
$$

We confirm these relationships for our example fit in the code cell below. We use the [`linalg`](https://numpy.org/doc/stable/reference/routines.linalg.html) linear algebra library in SciPy to compute the matrix inverse ([`inv`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html)). We also use the NumPy function [`array_str`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.array_str.html) to format each matrix element to four digits of precision, with `suppress_small=True` to represent the entries in decimal notation instead of scientific notation. Compare with the curvature matrix $\mathbf{A}$ at the bottom of p.&nbsp;95 and the error matrix $\mathbf{C}$ at the top of p.&nbsp;96 in *MU*—note that our column order is reversed from theirs because our `model` function lists the parameters in order (`m`,`c`).

In [ ]:
from scipy import linalg

Amm = np.sum(frequency**2 / err**2)
Amc = np.sum(frequency / err**2)
Acc = np.sum(1 / err**2)

A = np.array([[Amm, Amc], [Amc, Acc]])
Ainv = linalg.inv(A)

rho_mc = pCov[0, 1] / (mAlpha * cAlpha)

print("Curvature matrix:")
print(np.array_str(A, precision=4, suppress_small=True))
print()
print("Inverse curvature matrix:")
print(np.array_str(Ainv, precision=4, suppress_small=True))
print()
print("Covariance matrix:")
print(np.array_str(pCov, precision=4, suppress_small=True))
print()
print(f"Correlation coefficient: {rho_mc:.3f}")
print()

### Interpreting confidence limits

We saw in [NB 2.0](2.0-Basic-statistics.ipynb#How-to-interpret-the-mean-±-the-standard-error) that the mean and the standard error will both fluctuate with each new measurement, because we determine both from the data. On average, we expect the true mean of the distribution to lie within the confidence limit $\bar{x}\pm\hat{\alpha}$ in about 68 % of a set of repeated measurements. The code cell below shows how this generalizes to the case of two or more parameters. It simulates 25 sets of measurements like that shown in *MU* Fig.&nbsp;6.1(a) and shows the true values (red marker), the best-fit values (black diamonds), confidence contours for $\Delta\chi^2 = 1, 2.71, 4$, and $9$ (black curves), and the 68 % confidence limits for $m$ and $c$ (vertical and horizontal lines, respectively). In simulations (1) and (2), the true value lies outside both $\hat{m}\pm\alpha_{\hat{m}}$ and $\hat{c}\pm\alpha_{\hat{c}}$. The true value lies inside the $\hat{m}\pm\alpha_{\hat{m}}$ confidence interval and at the edge of the $\hat{c}\pm\alpha_{\hat{c}}$ confidence interval for simulation (3), and it lies within both $\hat{m}\pm\alpha_{\hat{m}}$ and $\hat{c}\pm\alpha_{\hat{c}}$ for simulation (9). The second figure shows the data and the fit for simulation (25), for reference. 

In [ ]:
trial = np.arange(1, 77)
noiseScale = 0.1

mRange = [0.925, 1.075]
cRange = [-0.35, 0.35]
m = np.linspace(mRange[0], mRange[1], Nm)
c = np.linspace(cRange[0], cRange[1], Nc)

chi2grid_trial = np.zeros([Nc, Nm])

fig, ax = plt.subplots(5, 5, sharex=True, sharey=True, figsize=[12.8, 9.6])

rg = default_rng(0)
for i in range(5):
    for j in range(5):
        meanVal = trial + noiseScale * trial * rg.normal(size=np.size(trial))
        pHat_trial, pCov_trial = np.polyfit(trial, meanVal, 1, w=1 / (noiseScale * trial), cov="unscaled")
        mHat_trial = pHat_trial[0]
        cHat_trial = pHat_trial[1]
        mAlpha_trial = np.sqrt(pCov_trial[0, 0])
        cAlpha_trial = np.sqrt(pCov_trial[1, 1])
        ax[i][j].plot(mHat_trial, cHat_trial, "kD")
        ax[i][j].plot(1, 0, "ro")
        ax[i][j].set_xlim([mRange[0], mRange[-1]])
        ax[i][j].set_xticks([0.95, 1.0, 1.05])
        ax[i][j].set_ylim([cRange[0], cRange[1]])
        ax[i][j].set_yticks([-0.3, 0.0, 0.3])

        def chi2fun(m, c):
            return np.sum((meanVal - (m * trial + c)) ** 2 / (noiseScale * trial) ** 2)

        for p in range(Nc):
            for q in range(Nm):
                chi2grid_trial[p, q] = chi2fun(m[q], c[p])

        chi2min = chi2fun(mHat_trial, cHat_trial)
        level_low = [*chi2min, 1, 2.7, 4, 9]
        ax[i][j].contour(
            m,
            c,
            chi2grid_trial,
            level_low,
            colors="k",
            linewidths=1,
            linestyles=["solid", "dashed", "dotted", "dotted"],
        )
        ax[i][j].hlines(cHat_trial + cAlpha_trial * np.array([-1, 1]), m[0], m[-1], linewidths=1, linestyles="dotted")
        ax[i][j].vlines(mHat_trial + mAlpha_trial * np.array([-1, 1]), c[0], c[-1], linewidths=1, linestyles="dotted")
        ax[i][j].text(1.05, 0.25, f"({5*i+j+1:d})")

fig.text(0.5, 0.04, "Gradient (trial/mean)", ha="center")
fig.text(0.04, 0.5, "Intercept (trial)", va="center", rotation="vertical")

plt.show()

plt.errorbar(trial, meanVal, yerr=noiseScale * trial, fmt=".")
plt.plot(trial, mHat_trial * trial + cHat_trial, "r-")
plt.xlabel("Trial")
plt.ylabel("Mean value")
plt.show()

## Exercise 1

In the Markdown cell below, assign each of the 25 simulated experiments in the figure above to one of the four categories listed (examples of each are already included). Note that both $\hat{m}$ and $\hat{c}$ are consistent with the true value in experiment (9), so it is listed in three separate categories. When you are done, tally the results and enter the fraction in the space provided, and discuss your results briefly.

*Response to Exercise 1*

* Consistent $\hat{m}$ ($1\sigma$):   3, 4, 6, 7, 9
* Consistent $\hat{c}$ ($1\sigma$):   5, 9
* Consistent $\hat{m}, \hat{c}$ ($1\sigma$):   9
* Inconsistent $\hat{m}, \hat{c}$ ($1\sigma$):   1, 2, 8


* Fraction consistent $\hat{m}$  ($1\sigma$):
* Fraction consistent $\hat{c}$ ($1\sigma$):   
* Fraction consistent $\hat{m}, \hat{c}$ ($1\sigma$):   
* Fraction inconsistent $\hat{m}, \hat{c}$ ($1\sigma$):   

*Discuss your results briefly here. Is it what you expect?*

The following code cell shows a scatterplot of $(\hat{m}, \hat{c})$ (blue dots) for `N_sim = 500` simulated measurements. A red marker indicates the true value, $(1, 0)$. The last simulation for $(\hat{m}, \hat{c})$ is shown with a black diamond instead of a blue dot, with $\Delta\chi^2 = 1, 2.71, 4$, and $9$ contours also shown for that fit. On average, the fits yield $(\hat{m}, \hat{c})$ that are centered on the true value and exhibit an elliptical distribution that resembles the shape of the countour levels.

In [ ]:
N_sim = 500
trial = np.arange(1, 77)
noiseScale = 0.1

mHat_sim = np.zeros(N_sim)
cHat_sim = np.zeros(N_sim)

rg = default_rng(0)
for i in range(N_sim):
    meanVal = trial + noiseScale * trial * rg.normal(size=np.size(trial))
    pHat_sim, pCov_sim = np.polyfit(trial, meanVal, 1, w=1 / (noiseScale * trial), cov="unscaled")
    mHat_sim[i] = pHat_sim[0]
    cHat_sim[i] = pHat_sim[1]


Nm = 50
Nc = 50

mRange = [0.95, 1.06]
cRange = [-0.35, 0.35]
m = np.linspace(mRange[0], mRange[1], Nm)
c = np.linspace(cRange[0], cRange[1], Nc)

# Define the chi-squared function for m and c, given the data


def chi2fun(m, c):
    normres = (meanVal - (m * trial + c)) / (noiseScale * trial)
    return np.sum(normres**2)


chi2grid_sim = np.zeros([Nc, Nm])
for i in range(Nc):
    for j in range(Nm):
        chi2grid_sim[i, j] = chi2fun(m[j], c[i])

chi2min_sim = chi2fun(mHat_sim[-1], cHat_sim[-1])
level = chi2min_sim + np.array([1, 2.71, 4, 9])

plt.plot(mHat_sim[0:-2], cHat_sim[0:-2], ".")
plt.plot(mHat_sim[-1], cHat_sim[-1], "kD", ms=10)
plt.plot(1, 0, "ro", ms=10)
plt.contour(m, c, chi2grid_sim, level, colors="k", linewidths=1, linestyles=["solid", "dashed", "dotted", "dotted"])
plt.xlabel("Gradient (mean/trial)")
plt.ylabel("Intercept (mean)")
plt.show()

## The error matrix for a linear fit

The matrix representation provides a convenient way to do error propagation. For example, we can rewrite *MU* Eq. (7.28) as

\begin{align}
\sigma_Z^2 &= \left(\frac{\partial Z}{\partial A}\right)^2\sigma_A^2 + 2\left(\frac{\partial Z}{\partial A}\right)\left(\frac{\partial Z}{\partial B}\right)\sigma_{AB} + \left(\frac{\partial Z}{\partial B}\right)^2\sigma_B^2\\
&= \begin{bmatrix}\frac{\partial Z}{\partial A} & \frac{\partial Z}{\partial B}\end{bmatrix}\begin{bmatrix}\sigma_A^2 & \sigma_{AB} \\ \sigma_{AB} & \sigma_B^2\end{bmatrix}\begin{bmatrix}\frac{\partial Z}{\partial A} \\ \frac{\partial Z}{\partial B}\end{bmatrix}.
\end{align}

For a linear fit $y = mx + b$, $\partial y/\partial m = x$ and $\partial y/\partial b = 1$, so we can use the covariance matrix $\mathbf{C}$ given by the fit to estimate the uncertainty in the *functional estimate* $\hat{y}$ at a given $x$,

\begin{align}
\alpha_\hat{y}^2 &= \begin{bmatrix}x & 1\end{bmatrix}\begin{bmatrix}C_{mm} & C_{mc} \\ C_{mc} & C_{cc}\end{bmatrix}\begin{bmatrix}x \\ 1\end{bmatrix}\\
&= C_{mm} x^2 + 2C_{mc}x + C_{cc}.
\end{align}

Note that $\mathbf{C}$ is determined from the fit to the data, so it represents a *particular sample* from a statistical distribution that describes all similar experiments. Likewise, our result for $\alpha_\hat{y}$ will also depend on the data. In contrast, our error propagation calculation involved *known, fixed parameters* $\sigma_Z, \sigma_A, \sigma_B$, and $\sigma_{AB}$ for an assumed multivariate Gaussian *parent distribution*. Our estimate $\alpha_\hat{y}$ essentially replaces the fixed but unknown parent distribution parameters with the mean values $\{\hat{m}, \hat{b}\}$ and covariance matrix $\mathbf{C}$, which are known but will fluctuate from one experiment to the next.

The quantity $\alpha_\hat{y}$ represents the uncertainty in the *model*, in contrast with the $\alpha_y$ (without the hat on $y$) that we determine from a sample of $y$ *data*. In general these are not the same—for example, we can define $\alpha_\hat{y}$ (with the hat) at an arbitrary value of $x$, even where we have no measurements. Moreover, $\alpha_y$ (without the hat) indicates the uncertainty in *a particular measurement* of $y$, whereas $\alpha_\hat{y}$ (with the hat) exploits information *from all measurements* of $y$—assuming, of course, that the model correctly describes the data in the first place!

The code cell below plots the fit with the 1*&sigma;* functional uncertainty bounds obtained from the fit covariance matrix `pCov`. We extrapolate the fit up to $x = 150$&nbsp;Hz, well beyond the data, to show that the uncertainty bounds increase as the frequency increases beyond the upper data limit—the bounds approach $(\hat{m}\pm\alpha_{\hat{m}})x + \hat{c}$ asymptotically with $|x|\rightarrow\infty$, and they are minimized near the center of the data range. We use the NumPy [`stack`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.stack.html) function to create the `2 x N` array,

$$
\nabla_{m,b}\mathbf{y} = \begin{bmatrix} x_1 & x_2 & \ldots & x_N\\ 1 & 1 & \ldots & 1\end{bmatrix},
$$

then compute the uncertainty $\alpha_\hat{f}(x_i)$ by computing the following for each column $\nabla_{m,b}y(x_i)$ of $\nabla_{m,b}\mathbf{y}$:

$$
\alpha_\hat{f}(x_i) = \sqrt{[\nabla_{m,b}y(x_i)]^\intercal\,\mathbf{C}\,[\nabla_{m,b}y(x_i)]}.
$$

Note that NumPy uses the `@` character as the matrix multiplication operation.

In [ ]:
# Define frequency array for displaying the model
N = 1000
fModel = np.linspace(0, 150, N)

gradf = np.stack((fModel, np.ones(np.shape(fModel))))
alphafhat = np.zeros(np.shape(fModel))
for i in np.arange(N):
    alphafhat[i] = np.sqrt(np.transpose(gradf[:, i]) @ pCov @ gradf[:, i])

# Set initial parameters m0 and b0
mInit = 2
bInit = 0

# Define model function


def model(x, m, b):
    return m * x + b


fMean = model(fModel, mInit, bInit)
fUpper = fMean + alphafhat
fLower = fMean - alphafhat

# Make the plot
plt.plot(fModel, fMean, "r-")
plt.plot(fModel, fUpper, "c--")
plt.plot(fModel, fLower, "c--")
plt.errorbar(frequency, voltage, yerr=err, fmt="ko")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Voltage (mV)")
plt.title("Data with linear model, initial parameters")
plt.xlim(fModel[0], fModel[-1])
plt.ylim(0, 2 * fModel[-1])
plt.show()

## Summary

Here is a list of what you should be able to do after completing this notebook.

* Describe the meaning and significance of the error surface.
* Construct an error surface for a theoretical model of experimental data.
* Describe the meaning and significance of the covariance matrix for a fit.
* Describe the relationship between the covariance matrix for a fit and the curvature matrix of the associated error surface.
* Use the covariance matrix from a fit to estimate the parameter uncertainties.
* Explain how to interpret parameter uncertainties obtained from a fit.

##### About this notebook
© J. Steven Dodge, 2020. Available from [SFU Physics GitLab](https://gitlab.phys.sfu.ca/physcrs/course-material-phys233).  The notebook text is licensed under CC BY 4.0. See more at [Creative Commons](https://creativecommons.org/licenses/by/4.0/). The notebook code is open source under the [MIT License](https://opensource.org/licenses/MIT).